In [24]:
TRADE_PAGE_URL = "https://www.capitoltrades.com/trades?per_page=96&assetType=stock&page="
TRADE_PAGE_TOTAL_PAGES = 5

In [1]:
import requests
from bs4 import BeautifulSoup
import sqlite3

# Function to create a SQLite database and table
def create_database():
    conn = sqlite3.connect('trades_data.db')
    cursor = conn.cursor()

    # Create a table if not exists
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS trades (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            politician TEXT,
            company TEXT,
            kind TEXT,
            publishtime TEXT,
            tradedtime TEXT,
            timetofill  TEXT,
            size TEXT,
            targetprice  TEXT
        )
    ''')

    conn.commit()
    conn.close()

# Function to parse and insert data into the database
def insert_data(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    table = soup.find('table', class_='q-table trades-table')

    if table:
        rows = table.find_all('tr')[1:]  # Skip the header row

        conn = sqlite3.connect('trades_data.db')
        cursor = conn.cursor()

        for row in rows:
            columns = row.find_all('td')
            print()
            politician = columns[0].text.strip()
            company = columns[1].text.strip()
            kind = columns[2].text.strip()
            publishtime = columns[3].text.strip()
            tradedtime = columns[4].text.strip()
            timetofill = columns[5].text.strip()
            size = columns[6].text.strip()
            targetprice = columns[7].text.strip()

            print(  columns[5].text.strip(),
             columns[6].text.strip(),
             columns[7].text.strip(),
              columns[8].text.strip())
            # Insert data into the table
            cursor.execute('''
                INSERT INTO trades (politician, company, kind, publishtime, tradedtime, timetofill, size, targetprice)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            ''', (politician, company, kind, publishtime, tradedtime, timetofill, size, targetprice))

        conn.commit()
        conn.close()


def read_data(politician=None , company=None, kind=None, publishtime=None, tradedtime=None, timetofillv=None, size=None, targetprice=None):
    conn = sqlite3.connect('trades_data.db')
    cursor = conn.cursor()

    cursor.execute('''
            SELECT * FROM trades
            WHERE  politician = ? AND company = ? AND kind = ? AND  publishtime = ? AND  tradedtime = ? AND timetofill= ? AND  size= ? AND targetprice= ? AND 
        ''',  (politician, company, kind, publishtime, tradedtime, timetofill, size, targetprice))

    return cursor.fetchall()

def read_data_all():
    conn = sqlite3.connect('trades_data.db')
    cursor = conn.cursor()

    cursor.execute("SELECT * FROM TRADES")

    return cursor.fetchall()

def drop(tablename=None):
    conn = sqlite3.connect('trades_data.db')
    cursor = conn.cursor()

    cursor.execute("DROP TABLE {tablename} CASCADE")

    return cursor.fetchall()


def populate_database():
    create_database()
    for page_num in range(1, TRADE_PAGE_TOTAL_PAGES + 1):
        url = TRADE_PAGE_URL + str(TRADE_PAGE_TOTAL_PAGES)
        html_content = fetch_data(url)

        if html_content:
            insert_data(html_content)
            print(f"Page {page_num} processed successfully.")

    

# Function to fetch data from the specified URL
def fetch_data(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        print(f"Failed to fetch data from {url}")
        return None

# Main loop to iterate through pages and extract data
def main():

    while True:
            print("\nMain Menu:")
            print("1. Create/Refresh Database")
            print("2. Read Data from Database")
            print("3. Exit")
            print("4. Drop")
            print("5. Read All Data from Database")

            choice = input("Enter your choice (1, 2, 3 or 4): ")

            if choice == '1':
                populate_database()
                print("Database created/refreshed successfully.")

            elif choice == '2':
                symbol = input("Enter symbol (optional): ")
                company = input("Enter company (optional): ")
                price = input("Enter price (optional): ")
                volume = input("Enter volume (optional): ")
                timestamp = input("Enter timestamp (optional): ")

                data = read_data(symbol, company, price, volume, timestamp)
                print("\nResult:")
                for row in data:
                    print(row)
            elif choice == '5':
                print(read_data_all())

            elif choice == '3':
                print("Exiting program. Goodbye!")
                break

            elif choice =='4':
                tablename = input("Enter volume (optional): ")
                drop(tablename)
                print("")

            else:
                print("Invalid choice. Please enter 1, 2, or 3, 4 ou 5.")


if __name__ == "__main__":
    main()


Main Menu:
1. Create/Refresh Database
2. Read Data from Database
3. Exit
4. Drop
5. Read All Data from Database


OperationalError: near "TABLEtradesCASCADE": syntax error